<a href="https://colab.research.google.com/github/sonu8499/ClassifyMeister-/blob/main/BOPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [6]:
import pandas as pd
import numpy as np
import math as math
import yfinance as yf

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df = pd.read_csv('Nifty_50_Data.csv')
df.head(50)

In [7]:
def volatility(df):
  df['volat'] = df['Adj Close'].pct_change()
  vol = df['volat'].std(ddof=0)*np.sqrt(252)
  return vol

In [11]:
volatility(df)

NameError: ignored

In [10]:
df.head(50)

NameError: ignored

In [ ]:
v = volatility(df)

In [ ]:
def alphaProbability(r,dt,v):
  u = np.exp(v*np.sqrt(dt))
  d = 1/u
  rnp = (np.exp(r*dt)-d)/(u-d)
  return rnp

In [ ]:
S = 17875

In [ ]:
def bopm(n,t,S,v):
  dt = t/n
  u = np.exp(v*np.sqrt(dt))
  d = 1/u
  Pm = np.zeros((n+1, n+1))
  for j in range(n+1):
    for i in range(j+1):
      Pm[i,j] = S*math.pow(d,i) * math.pow(u,j-i)
  return Pm

In [ ]:
n = 5
t = 200/365
v = volatility(df)
x = bopm(n,t,S,v)

print('n = 5:\n',np.matrix(x.astype(int)))

n = 5:
 [[17875 18925 20038 21216 22464 23784]
 [    0 16882 17875 18925 20038 21216]
 [    0     0 15945 16882 17874 18925]
 [    0     0     0 15059 15945 16882]
 [    0     0     0     0 14223 15059]
 [    0     0     0     0     0 13433]]


In [ ]:
def OptionsVal(n,S,k,r,v,T,PC):
  dt = T/n
  u = math.exp(v*math.sqrt(dt))
  d = 1/u
  p = (math.exp(r*dt)-d)/(u-d)
  Pm = np.zeros((n+1,n+1))
  Cm = np.zeros((n+1,n+1))
  tmp = np.zeros((2,n+1))
  for j in range(n+1):
    tmp[0,j] = S*math.pow(d,j)
    tmp[1,j] = S*math.pow(u,j)
  tot = np.unique(tmp)
  c = n
  for i in range(c+1):
    for j in range(c+1):
      Pm[i,j-c-1] = tot[(n-i)+j]
    c=c-1
  for j in range(n+1,0,-1):
    for i in range(j):
      if(PC == 1):
        if(j == n+1):
          Cm[i,j-1] = max(k-Pm[i,j-1],0)
        else:
          Cm[i,j-1] = math.exp(-r*dt)*(p*Cm[i,j] + (1-p)*Cm[i+1,j])
      if(PC == 0):
        if(j == n+1):
          Cm[i,j-1] = max(Pm[i,j-1]-k,0)
        else:
          Cm[i,j-1] = math.exp(-r*dt)*(p*Cm[i,j] + (1-p)*Cm[i+1,j])
  return [Pm,Cm]

In [ ]:
k = S
r = 0.06
T = 20/36
n = 10
PC = 0
Pm,CmC = OptionsVal(n,S,k,r,v,T,PC)
PC = 1
_,CmP = OptionsVal(n,S,k,r,v,T,PC)
print('Pricing:\n',np.matrix(Pm.astype(int)))
print('Call Option:\n',np.matrix(CmC.astype(int)))
print('Put Option:\n',np.matrix(CmP.astype(int)))

Pricing:
 [[17875 18617 19389 20194 21033 21906 22815 23762 24749 25776 26847]
 [    0 17162 17875 18617 19389 20194 21033 21906 22815 23762 24749]
 [    0     0 16478 17162 17875 18617 19389 20194 21033 21906 22815]
 [    0     0     0 15821 16478 17162 17875 18617 19389 20194 21033]
 [    0     0     0     0 15190 15821 16478 17162 17875 18617 19389]
 [    0     0     0     0     0 14585 15190 15821 16478 17162 17875]
 [    0     0     0     0     0     0 14004 14585 15190 15821 16478]
 [    0     0     0     0     0     0     0 13445 14004 14585 15190]
 [    0     0     0     0     0     0     0     0 12909 13445 14004]
 [    0     0     0     0     0     0     0     0     0 12395 12909]
 [    0     0     0     0     0     0     0     0     0     0 11901]]
Call Option:
 [[1201 1631 2164 2799 3526 4326 5177 6065 6993 7961 8972]
 [   0  723 1040 1460 1995 2646 3394 4209 5059 5947 6874]
 [   0    0  370  572  865 1274 1818 2497 3277 4090 4940]
 [   0    0    0  145  245  408  667 1062 